In [1]:
import mongoengine
from datetime import datetime
import pandas as pd
import numpy as np
import uuid
import json

In [2]:
cd Github/placemaker-flask/placemaker/src/placemaker/

/Users/Rishi/GitHub/placemaker-flask/placemaker/src/placemaker


In [3]:
from models import *
from settings import *

In [4]:
mongoengine.connect(MONGO_DB, host=MONGO_HOST)

MongoClient('db.clvr.tech', 27017)

# Creating St. Louis Continuum of Care Parent Document

In [ ]:
stlouis = Coc(coc_id = uuid.uuid4(), url = "https://www.stlouis-mo.gov/government/departments/human-services/homeless-services/continuum-of-care/")
stlouis.save()

# Filling Person Collection with HUD Compliant Legacy Data

In [5]:
clients = pd.read_csv('/Users/Rishi/Downloads/Sample Dataset - Client.csv')
disabilities = pd.read_csv('/Users/Rishi/Downloads/Sample Dataset - Disabilities.csv')
education = pd.read_csv('/Users/Rishi/Downloads/Sample Dataset - EmploymentEducation.csv')
enrollment = pd.read_csv('/Users/Rishi/Downloads/Sample Dataset - Enrollment.csv')
exit = pd.read_csv('/Users/Rishi/Downloads/Sample Dataset - Exit.csv')
funder = pd.read_csv('/Users/Rishi/Downloads/Sample Dataset - Funder.csv')
health_dv = pd.read_csv('/Users/Rishi/Downloads/Sample Dataset - HealthAndDV.csv')

In [6]:
for indx, row in clients.iterrows():
    person = Person()
    
    # Name
    name_info = NameInfo()
    
    if row['Name_Data_Quality'] == 1:
        name_info.name_type = name_info.name_tuples[0]
        if not pd.isnull(row['First_Name']):
            name_info.first_name = row['First_Name']
        if not pd.isnull(row['Last_Name']): 
            name_info.last_name = row['Last_Name']
        if not pd.isnull(row['Middle_Name']):
            name_info.middle_name = row['Middle_Name']
    elif row['Name_Data_Quality'] == 2:
        name_info.name_type = name_info.name_tuples[1]
        if not pd.isnull(row['First_Name']):   
            name_info.first_name = row['First_Name']
        if not pd.isnull(row['Middle_Name']):
            name_info.middle_name = row['Middle_Name']
        if not pd.isnull(row['Last_Name']):
            name_info.last_name = row['Last_Name']
    elif row['Name_Data_Quality'] == 8:
        name_info.name_type = name_info.name_tuples[2]
    elif row['Name_Data_Quality'] == 99:
        name_info.name_type = name_info.name_tuples[3]
    person.name_info = name_info
    
    # SSN
    ssn_info = SSNInfo()
    if row['SSNDataQuality'] == 1:
        ssn_info.ssn_type = ssn_info.ssn_tuples[0]
        ssn_info.ssn = str(int(row['SSN']))
    elif row['SSNDataQuality'] == 2:
        ssn_info.ssn_type = ssn_info.ssn_tuples[1]
        ssn_info.ssn = str(int(row['SSN']))
    elif row['SSNDataQuality'] == 8:
        ssn_info.ssn_type = ssn_info.ssn_tuples[2]
        ssn_info.ssn = None
    elif row['SSNDataQuality'] == 9:
        ssn_info.ssn_type = ssn_info.ssn_tuples[3]
        ssn_info.ssn = None
    elif row['SSNDataQuality'] == 99:
        ssn_info.ssn_type = ssn_info.ssn_tuples[4]
        ssn_info.ssn = None
    person.ssn_info = ssn_info
    
    # DOB
    dob_info = DOBInfo()
    if row['DOBDataQuality'] == 1:
        dob_info.dob_type = dob_info.dob_type_tuples[0]
        dob_info.dob = datetime.strptime(row['DOB'], '%m/%d/%Y')
    elif row['DOBDataQuality'] == 2:
        dob_info.dob_type = dob_info.dob_type_tuples[1]
    elif row['DOBDataQuality'] == 8:
        dob_info.dob_type = dob_info.dob_type_tuples[2]
    elif row['DOBDataQuality'] == 99:
        dob_info.dob_type = dob_info.dob_type_tuples[3]
    else:
        dob_info.dob_type = dob_info.dob_type_tuples[4]
    person.dob_info = dob_info
    
    # Race
    for race, boolean in row[['AmIndAKNative','Asian', 'Black', 'NativeHIOtherPacific', 'White', 'RaceNone']].iteritems():
        if boolean:
            if race == 'AmIndAKNative':
                person.race = person.race_tuples[0]
            if race == 'Asian':
                person.race = person.race_tuples[1]
            if race == 'Black':
                person.race = person.race_tuples[2]
            if race == 'NativeHIOtherPacific':
                person.race = person.race_tuples[3]
            if race == 'White':
                person.race = person.race_tuples[4]
            if race == 'RaceNone':
                person.race = person.race_tuples[7]
    
    # Ethnicity
    # nothing in dataset for ethnicity
    person.ethnicity = person.ethnicity_tuples[4]
    
    # Gender
    gender_info = GenderInfo()
    if row['OtherGender'] == 1:
        gender_info.gender = gender_info.gender_tuples[4]
    elif row['Gender'] == 0:
        gender_info.gender = gender_info.gender_tuples[0]
    elif row['Gender'] == 1:
        gender_info.gender = gender_info.gender_tuples[1]
    elif row['Gender'] == 2:
        gender_info.gender = gender_info.gender_tuples[2]
    elif row['Gender'] == 3:
        gender_info.gender = gender_info.gender_tuples[3]
    elif row['Gender'] == 8:
        gender_info.gender = gender_info.gender_tuples[5]
    elif row['Gender'] == 9:
        gender_info.gender = gender_info.gender_tuples[6]
    elif row['Gender'] == 99:
        gender_info.gender = gender_info.gender_tuples[7]
    person.gender_info = gender_info
    
    # Veteran Status
    if row['VeteranStatus'] == 0:
        person.veteran = person.veteran_status_tuples[0]
    elif row['VeteranStatus'] == 1:
        person.veteran = person.veteran_status_tuples[1]
    elif row['VeteranStatus'] == 8:
        person.veteran = person.veteran_status_tuples[2]
    elif row['VeteranStatus'] == 9:
        person.veteran = person.veteran_status_tuples[3]
    elif row['VeteranStatus'] == 99:
        person.veteran = person.veteran_status_tuples[4]
    
    # Disabling Condition
    disabling_condition_info = DisablingConditionInfo()
    if row['UUID'] in enrollment['PersonalID'].unique():
        if enrollment[enrollment['PersonalID'] == row['UUID']]['DisablingCondition'].tail(1).notnull().reset_index().iloc[0, 1]:
            if enrollment[enrollment['PersonalID'] == row['UUID']]['DisablingCondition'].tail(1).reset_index().iloc[0,1] == 0:
                disabling_condition_info.disabling_condition = disabling_condition_info.disabling_condition_tuples[0]
            elif enrollment[enrollment['PersonalID'] == row['UUID']]['DisablingCondition'].tail(1).reset_index().iloc[0,1] == 1:
                disabling_condition_info.disabling_condition = disabling_condition_info.disabling_condition_tuples[1]
            elif enrollment[enrollment['PersonalID'] == row['UUID']]['DisablingCondition'].tail(1).reset_index().iloc[0,1] == 8:
                disabling_condition_info.disabling_condition = disabling_condition_info.disabling_condition_tuples[2]
            elif enrollment[enrollment['PersonalID'] == row['UUID']]['DisablingCondition'].tail(1).reset_index().iloc[0,1] == 9:
                disabling_condition_info.disabling_condition = disabling_condition_info.disabling_condition_tuples[3]
            elif enrollment[enrollment['PersonalID'] == row['UUID']]['DisablingCondition'].tail(1).reset_index().iloc[0,1] == 99:
                disabling_condition_info.disabling_condition = disabling_condition_info.disabling_condition_tuples[4]
        else:
            disabling_condition_info.disabling_condition = disabling_condition_info.disabling_condition_tuples[4]
            
    else:
        disabling_condition_info.disabling_condition = disabling_condition_info.disabling_condition_tuples[4]
    person.disabling_condition_info = disabling_condition_info
    
    # Living Situation
    homeless_indx_array = [(16,0),(1,1),(18,2),(27,3)]
    institutional_indx_array = [(15,4),(6,5),(7,6),(24,7),(4,8),(5,9)]
    transitional_indx_array =  [(14,10),(23,11),(21,12),(3,13),(22,14),(19,15),(25,16),(20,17),(26,18),(12,19),(13,20),(2,21),(8,22),(9,23)]
    living_situation_info = LivingSituationInfo()
    
    # Living Situation: residence_type and residence_subtype
    if enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePrior'].tail(1).isnull().reset_index().iloc[0,1] or enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePrior'].tail(1).reset_index().iloc[0,1] == 99:
        living_situation_info.residence_type = living_situation_info.residence_type_tuples[3]
        living_situation_info.residence_subtype = living_situation_info.residence_subtype_tuples[24]
        
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePrior'].tail(1).reset_index().iloc[0,1] in [val[0] for val in homeless_indx_array]:
        living_situation_info.residence_type = living_situation_info.residence_type_tuples[0]
        for legacy_new_tuple in homeless_indx_array:
            if enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePrior'].tail(1).reset_index().iloc[0,1] == legacy_new_tuple[0]:
                living_situation_info.residence_subtype = living_situation_info.residence_subtype_tuples[legacy_new_tuple[1]]
    
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePrior'].tail(1).reset_index().iloc[0,1] in [val[0] for val in institutional_indx_array]:
        living_situation_info.residence_type = living_situation_info.residence_type_tuples[1]
        for legacy_new_tuple in institutional_indx_array:
            if enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePrior'].tail(1).reset_index().iloc[0,1] == legacy_new_tuple[0]:
                living_situation_info.residence_subtype = living_situation_info.residence_subtype_tuples[legacy_new_tuple[1]]
    
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePrior'].tail(1).reset_index().iloc[0,1] in [val[0] for val in transitional_indx_array]:
        living_situation_info.residence_type = living_situation_info.residence_type_tuples[2]
        for legacy_new_tuple in transitional_indx_array:
            if enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePrior'].tail(1).reset_index().iloc[0,1] == legacy_new_tuple[0]:
                living_situation_info.residence_subtype = living_situation_info.residence_subtype_tuples[legacy_new_tuple[1]]
    
    # Living Situation: current_length_of_stay
    if enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 10:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[0]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 11:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[1]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 2:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[2]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 3:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[3]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 4:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[4]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 5:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[5]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 8:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[6]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 9:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[7]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['ResidencePriorLengthOfStay'].tail(1).reset_index().iloc[0,1] == 99:
        living_situation_info.current_length_of_stay = living_situation_info.current_length_of_stay_tuples[8]
    
    # Living Situation: current_approx_start_date
    if enrollment[enrollment['PersonalID']==row['UUID']]['DateToStreetESSH'].tail(1).notnull().reset_index().iloc[0,1]:
        living_situation_info.current_approx_start_date = datetime.strptime(enrollment[enrollment['PersonalID']==row['UUID']]['DateToStreetESSH'].tail(1).reset_index().iloc[0,1], '%m/%d/%Y')
    
    # Living Situation: total_count
    if enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).isnull().reset_index().iloc[0,1] or enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 99 or enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 0:
        living_situation_info.total_count = living_situation_info.total_count_tuples[6]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 1:
        living_situation_info.total_count = living_situation_info.total_count_tuples[0]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 2:
        living_situation_info.total_count = living_situation_info.total_count_tuples[1]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 3:
        living_situation_info.total_count = living_situation_info.total_count_tuples[2]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 4:
        living_situation_info.total_count = living_situation_info.total_count_tuples[3]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 8:
        living_situation_info.total_count = living_situation_info.total_count_tuples[4]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['TimesHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 9:
        living_situation_info.total_count = living_situation_info.total_count_tuples[5]
    
    # Living Situation: total_months
    if enrollment[enrollment['PersonalID']==row['UUID']]['MonthsHomelessPastThreeYears'].tail(1).isnull().reset_index().iloc[0,1]:
        living_situation_info.total_months = living_situation_info.total_month_tuples[15]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['MonthsHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 8:
        living_situation_info.total_months = living_situation_info.total_month_tuples[13]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['MonthsHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 9:
        living_situation_info.total_months = living_situation_info.total_month_tuples[14]
    elif enrollment[enrollment['PersonalID']==row['UUID']]['MonthsHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] == 99:
        living_situation_info.total_months = living_situation_info.total_month_tuples[15]
    else:
        living_situation_info.total_months = living_situation_info.total_month_tuples[int(enrollment[enrollment['PersonalID']==row['UUID']]['MonthsHomelessPastThreeYears'].tail(1).reset_index().iloc[0,1] - 101)]
    
    # Living Situation: prior_residence_type
    
    
    # Living Situation: prior_residence_subtype
    # Living Situation: prior_approx_start_date
    
    person.living_situation_info = living_situation_info
    
    # Project Entry Date
    person.project_entry_date = datetime.strptime(enrollment[enrollment['PersonalID']==row['UUID']]['EntryDate'].tail(1).reset_index().iloc[0,1], '%m/%d/%Y')
    
    # Project Exit Date
    if len(exit[exit['PersonalID']==row['UUID']]) != 0:
        person.project_exit_date = datetime.strptime(exit[exit['PersonalID']==row['UUID']]['ExitDate'].tail(1).reset_index().iloc[0,1], '%m/%d/%Y')
    
    # Destination
    destination_info = DestinationInfo()
    destination_legacy_tuples_list = [(24, "Deceased"), 
                                      (1, "Emergency shelter, including hotel or motel paid for with emergency shelter voucher"),
                                      (15,"Foster care home or foster care group home"), (6,"Hospital or other residential non-psychiatric medical facility"),
                                      (14,"Hotel or motel paid for without emergency shelter voucher"),
                                      (7,"Jail, prison or juvenile detention facility"),
                                      (25,"Long-term care facility or nursing home"),
                                      (26,"Moved from one HOPWA funded project to HOPWA PH"),
                                      (27,"Moved from one HOPWA funded project to HOPWA TH"),
                                      (11,"Owned by client, no ongoing housing subsidy"),
                                      (21,"Owned by client, with ongoing housing subsidy"),
                                      (3,"Permanent housing for formerly homeless persons (such as: CoC project; or HUD legacy programs; or HOPWA PH)"),
                                      (16,"Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)"),
                                      (4,"Psychiatric hospital or other psychiatric facility"),
                                      (10,"Rental by client, no ongoing housing subsidy"),
                                      (19,"Rental by client, with VASH housing subsidy"),
                                      (28,"Rental by client, with GPD TIP housing subsidy"),
                                      (20,"Rental by client, with other ongoing housing subsidy"),
                                      (29,"Residential project or halfway house with no homeless criteria"),
                                      (18,"Safe Haven"),
                                      (22,"Staying or living with family, permanent tenure"),
                                      (12,"Staying or living with family, temporary tenure (e.g. room, apartment or house)"),
                                      (23,"Staying or living with friends, permanent tenure"),
                                      (13,"Staying or living with friends, temporary tenure (e.g. room apartment or house)"),
                                      (5,"Substance abuse treatment facility or detox center"),
                                      (2,"Transitional housing for homeless persons (including homeless youth)"),
                                      (17,"Other"),
                                      (30,"No exit interview completed"),
                                      (8,"Client doesn’t know"),
                                      (9,"Client refused"),
                                      (99,"Data not collected")]
    if len(exit[exit['PersonalID']==row['UUID']]['Destination']) != 0:  
        if exit[exit['PersonalID']==row['UUID']]['Destination'].tail(1).isnull().reset_index().iloc[0,1]:
            destination_info.destination = destination_info.destination_tuples[-1]
        else:
            for indx, legacy_tuple in enumerate(destination_legacy_tuples_list):
                if legacy_tuple[0] == exit[exit['PersonalID']==row['UUID']]['Destination'].tail(1).reset_index().iloc[0,1]:
                    destination_info.destination = destination_info.destination_tuples[indx]
                    break
    else:
        destination_info.destination = destination_info.destination_tuples[-1]
    person.destination_info = destination_info
    
    # Personal ID
    person.personal_id = uuid.uuid4()
    
    # Household ID
    person.family_id = int(enrollment[enrollment['PersonalID']==row['UUID']]['HouseholdID'].tail(1).reset_index().iloc[0,1])
    
    # Legacy ID
    person.legacy_id = int(row['UUID'])
    
    # Household Head Relationship
    person.household_head_relationship = person.household_head_relationship_tuples[enrollment[enrollment['PersonalID']==row['UUID']]['RelationshipToHoH'].tail(1).reset_index().iloc[0,1]-1]
    stlouis = Coc.objects[0]
    person.coc = stlouis
    # Client Location
    print person.to_json()
    person.save()

{"name_info": {"first_name": "Robert", "middle_name": "J", "last_name": "Hanks", "name_type": "Full name reported"}, "ssn_info": {"ssn": "100090077", "ssn_type": "Full SSN reported"}, "dob_info": {"dob": {"$date": -536457600000}, "dob_type": "Full DOB reported"}, "race": "Black or African American", "ethnicity": "Data not collected", "gender_info": {"gender": "Male"}, "veteran": "Yes", "disabling_condition_info": {"disabling_condition": "No"}, "living_situation_info": {"residence_type": "Literally Homeless", "residence_subtype": "Place not meant for habitation (e.g. a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)", "current_length_of_stay": "One month or more, but less than 90 days", "current_approx_start_date": {"$date": 1408665600000}, "total_count": "Two times", "total_months": "Data not collected"}, "project_entry_date": {"$date": 1413936000000}, "project_exit_date": {"$date": 1447372800000}, "destination_info": {"destination": "bus/train/sub

# Filling Organization Collection with  St. Louis Data

In [7]:
organizations = pd.read_csv('/Users/Rishi/Downloads/organizations.csv')

In [9]:
for indx, row in organizations.iterrows():
    organization = Organization()
    
    organization.name = row['Agency/Program Name']
    
    organization.organization_type = row['Organization Type']
    
    contact_info = ContactInfo()
    if pd.notnull(row['Contact Name']):
        contact_info.contact_name = row['Contact Name']
    if pd.notnull(row['Number']):
        contact_info.phone_number = row['Number']
    if pd.notnull(row['Email']):
        contact_info.email = row['Email']
    if pd.notnull(row['Contact Name']):
        contact_info.contact_name = row['Contact Name']
    if pd.notnull(row['Address']):
        contact_info.address = row['Address']
    if pd.notnull(row['City']):
        contact_info.city = row['City']
    if pd.notnull(row['State']):
        contact_info.state = row['State']
    if pd.notnull(row['Zip Code']):
        contact_info.zip_code = row['Zip Code']
    organization.contact_info = contact_info
    
    if pd.notnull(row['bed_capacity']):
        organization.bed_capacity = int(row['bed_capacity'])
        
    if pd.notnull(row['family_capacity']):
        organization.family_capacity = int(row['family_capacity'])
    
    eligibility_info = EligibilityInfo()
    if pd.notnull(row['individual_gender']):
        eligibility_info.individual_gender = row['individual_gender']
    if pd.notnull(row['transgender']):
        eligibility_info.transgender = row['transgender']
    if pd.notnull(row['household_size_min']):
        eligibility_info.family_size_min = int(row['household_size_min'])
    if pd.notnull(row['household_size_max']):
        eligibility_info.family_size_max = int(row['household_size_max'])
    if pd.notnull(row['veteran']):
        eligibility_info.veteran = row['veteran']
    if pd.notnull(row['pregnancy']):
        eligibility_info.pregnancy = row['pregnancy']
    if pd.notnull(row['sobriety']):
        eligibility_info.sobriety = row['sobriety']
    if pd.notnull(row['HIV']):
        eligibility_info.hiv = row['HIV']
    if pd.notnull(row['referral']):
        eligibility_info.referral = row['referral']
    if pd.notnull(row['hour_open']):
        eligibility_info.hour_open = row['hour_open']
    if pd.notnull(row['hour_close']):
        eligibility_info.hour_close = row['hour_close']
    if pd.notnull(row['expected_county']):
        eligibility_info.expected_county = row['expected_county']
    if pd.notnull(row['min_age']):
        eligibility_info.min_age = int(row['min_age'])
    if pd.notnull(row['max_age']):
        eligibility_info.max_age = int(row['max_age'])
    organization.eligibility_info = eligibility_info
    stlouis = Coc.objects[0]
    organization.coc = stlouis
    organization.save()

# Updating Person Collection with Additional Eligibility Categories

In [19]:
for indx, person in enumerate(Person.objects()):
    print indx
    print person
    print json.loads(person.to_json())

0
Person object
{u'living_situation_info': {u'total_months': u'Data not collected', u'residence_type': u'Transitional & Permanent Housing Situation', u'current_length_of_stay': u'One year or longer', u'residence_subtype': u'Rental by client, no ongoing housing subsidy', u'total_count': u'Data not collected'}, u'name_info': {u'first_name': u'Kent', u'last_name': u'Kiernan', u'middle_name': u'C', u'name_type': u'Full name reported'}, u'household_head_relationship': u'Self (head of household)', u'ssn_info': {u'ssn': u'100244633', u'ssn_type': u'Full SSN reported'}, u'veteran': u'Yes', u'coc': {u'$uuid': u'7abed393929e4a9fa61a14bc83f0c1f2'}, u'dob_info': {u'dob': {u'$date': 0}, u'dob_type': u'Full DOB reported'}, u'destination_info': {u'destination': u'Psychiatric hospital or other psychiatric facility'}, u'family_id': 62053, u'disabling_condition_info': {u'disabling_condition': u'No'}, u'race': u'White', u'project_exit_date': {u'$date': 1452729600000}, u'legacy_id': 244633, u'project_entr

In [20]:
people = [json.loads(person.to_json()) for person in Person.objects()]
person = people[0]
print len(['dummy' if person['family_id'] == iter_person['family_id'] else '' for iter_person in people].remove(''))

TypeError: object of type 'NoneType' has no len()

In [24]:

print ['dummy' if iter_person['family_id'] == 62053 else '' for iter_person in people].count('dummy')

7


In [26]:
x =range(0,5)

In [28]:
del x[2]

In [29]:
x

[0, 1, 3, 4]

In [30]:
if not False:
    print 'hi'

hi


In [33]:
datetime.now().minute

41

In [41]:
datetime.now().year - datetime(2000,1,1).year

16

In [38]:
datetime(2000,1,1).date

<function date>